# Team 16 Toyota SmartHome ML Training

## Setting up dependencies

#### Check CUDA Version
To check if CUDA is available, perform the following:<br>
1) Go to device manager
2) Expand Display adapters
3) Go to https://en.wikipedia.org/wiki/CUDA#GPUs_supported <br>(https://developer.nvidia.com/cuda-gpus has an outdated list e.g. GTX 1050 Ti is not included)
4) If exists, Open Command prompt and execute "nvidia-smi" or "nvcc --version"<br>
5) Install pytorch according the CUDA version displayed.<br>
<b>CUDA 10.2</b>: No longer available for windows<br>
<b>CUDA 11.3</b>: conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch<br>
<b>CUDA 11.6</b>: conda install pytorch torchvision torchaudio cudatoolkit=11.6 -c pytorch -c conda-forge<br>
<b>CPU</b>: conda install pytorch torchvision torchaudio cpuonly -c pytorch<br>

#### Additional Dependencies

Setup Virtual Environment
Run in Terminal:
- virtualenv .venv
- pip install -r requirements.txt

#### Importing Libraries

In [1]:
from ipywidgets import widgets, Button
from IPython.display import display, HTML, Video
from ipyfilechooser import FileChooser
import os
import io
import base64
from uploadData import UploadData

### Show/Hide code cells

By default, the code base in each cell will be hidden for secuirty purposes. However, you still can view the code if you clicked on the toggle button below to show or hide the code in the cell.

In [ ]:
HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Data Exploration

### File Upload

The following cell is to upload the file in the respective folder.
- Choose the files that you want to upload. The files can be in zip format.
- Click on the confirmation button to upload the file to the respective folder.

In [2]:
# Creates FileChooser and setting default directory (https://pypi.org/project/ipyfilechooser/)
fc = FileChooser('/Users')
fc.title = '<b>Select file to upload</b>'
# Restrict navigation to /Users
fc.sandbox_path = '/Users'
# Change hidden files
fc.show_hidden = False
# Customize dir icon
# fc.dir_icon = '/'
# fc.dir_icon_append = True

# Upload data function
def submit_upload(b):
    fn = fc.selected_filename
    identified_filetype, identified_file = UploadData.upload_data(fn, fc)
    with output:
        print(f"File: {identified_file}")
        print("File type identified to be/include: " + identified_filetype)

# Set input file types
# submit_upload() function in "function.py" sorts input for video, csv/xslx(Dataset)
confirmBtn = widgets.Button(
    description='Confirm Upload',
    tooltip='Confirm',
    button_style='success',
)
output = widgets.Output()
display(fc)
display(confirmBtn, output)
confirmBtn.on_click(submit_upload)


FileChooser(path='/Users', filename='', title='<b>Select file to upload</b>', show_hidden=False, select_desc='…

Button(button_style='success', description='Confirm Upload', style=ButtonStyle(), tooltip='Confirm')

Output()

### Displaying video with generated captions<br>
Here, we will look at selecting a video to generate the captions using the pre-trained model before displaying the video with the captions.

#### 1. Select a video from the Video Folder
- Choose a video from the directory.
- Click on the confirmation button to confirm the selection of the video.

In [ ]:
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc = FileChooser('./Data_Storage/Videos')
display(fc)

confirmBtn = widgets.Button(
    description='Confirm Selection',
    tooltip='Confirm',
    button_style='success',
)

# Confirm Video Selection
def submit_videoSelection(b):
    if fc.selected_filename is not None:
        with output:
            print(f"{fc.selected_filename} is selected")
    else:
        display(HTML('''
    <h3 style="color:red;">Error: Please select a video.</h3>'''))

fc.sandbox_path = './Data_Storage/Videos'
fc.show_hidden = False
fc.show_only_dirs = False
fc.title = '<b>Select input video</b>'
# fc.register_callback(play_video)
output = widgets.Output()
display(confirmBtn, output)
confirmBtn.on_click(submit_videoSelection)

#### 2. Play the selected video

- Click on the "Play Video" button to start play the video that you have selected previously.

In [ ]:
# read action and frame object from file
def readCaptionFile(filename,videoName):
    """
    Parameters
    ----------
    filename: the smarthome json file.
    videoName: The name of the selected video.
    
    Returns
    -------
    2d-array data
    """
    # Opening JSON file
    f = open(filename)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Iterating through the json
    # Closing file
    f.close()
    return data[videoName]["actions"]

# read reference text from txt file
def readReferenceFile(refFile):
    """
    Parameters
    ----------
    refFile: A txt file that contains all the label.
    
    Returns
    -------
    referenceDict
    """
    referenceDict = {}
    with open(refFile) as f:
        lines = f.readlines()
    for i in lines:
        x = i.split()
        referenceDict[str(x[0])] = x[1]
    return referenceDict

# create caption file
def formatCaptionFile(captionList, reference, captionPath):
    """
    Parameters
    ----------
    captionList: A list of caption.
    reference: A dict of reference derived from readReferenceFile function.
    captionPath: The path that stored all the caption.
    """
    start = "WEBVTT\n\n"
    captions = []
    for i in captionList:
        text = reference[str(i[0])]
        lines = convertFrameToTime(i[1]) + " --> " + convertFrameToTime(i[2]) + "\n" + text + "\n\n"
        captions.append(lines)
    f = open(captionPath, "w")
    f.write(start)
    f.writelines(captions)
    f.close()

# convert frame to time
def convertFrameToTime(frame):
    """
    Parameters
    ----------
    frame: The frame of the video.
    
    Returns
    -------
    The time after the conversion of frame to time.
    """
    seconds = int(frame/25)
    minutes = "00"
    if seconds >= 60:
        minutes = str(seconds // 60)
        seconds = seconds % 60
    if len(minutes) == 1:
        minutes = "0" + minutes
    seconds = str(seconds)
    #may need handle hour
    if len(seconds) == 1:
        seconds = "0" + seconds 
    return (minutes + ":" + seconds + ".000")

play_btn = widgets.Button(
    description='Play Video',
    tooltip='Confirm',
    button_style='primary',
)

# Play video function
def play_video(b):
    with output:
        print(f"Start playing {fc.selected_filename} ...........")
    fn = fc.selected_filename
    fn = fn.split('.')
    src = "./Data_Storage/Videos/" + fc.selected_filename
    video = io.open(src, 'r+b').read()
    encoded = base64.b64encode(video)
    #location of reference are place at root
    ref = readReferenceFile('./Data_Storage/all_labels.txt')
    # may need change the caption path to dynamic
    
    caption_location = './Data_Storage/Captions/' + fn[0] + ".vtt"
    # model result file should be some directory, here using root 
    captionList = readCaptionFile('./Data_Storage/smarthome_CS_51.json', fn[0])
    formatCaptionFile(captionList, ref, caption_location)
    display(HTML(data='''<video width="650" height="360" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        <track kind="captions" src={1} srclang="en" label="English" default>
        </video>'''.format(encoded.decode('ascii'), caption_location)))
    
output = widgets.Output()
display(play_btn, output)
play_btn.on_click(play_video)

## Inference

### Select Testing

The following codes will allow the users to select the model for inference.
- Choose and select a model in from the respective model folder.

In [4]:
mtpList = ['Pretrained_Models', 'Trained_Models']
mtp = widgets.Dropdown( #Model Type Picker
    options = mtpList,
    default = 0,
    description = 'Model Type',
)

# Select model function
def display_inference_model(f):
    rs = ptmodel.selected_path + ptmodel.selected_filename
    with selectedptmodel:
        print(ptmodel.selected_filename + " model has been selected.")

selectedptmodel = widgets.Output()
ptmodel = FileChooser('./Data_Storage/'+ mtp.value)
ptmodel.sandbox_path = './Data_Storage/'+ mtp.value
ptmodel.show_hidden = False
ptmodel.filter_pattern = ['*']
ptmodel.show_only_dirs = False
ptmodel.title = '<b>Select Inference Model</b>'
ptmodel.register_callback(display_inference_model)
display(ptmodel)
display(selectedptmodel)
# run = f"python test.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train False -num_channel 512 -lr 0.0002 -kernelsize 3 -APtype map -batch_size 1 -comp_info TSU_CS_RGB_PDAN -load_model {model_src}"
# % {run}



FileChooser(path='/Users/junjie/Documents/3104/new_project/ict3104-team16-2022/Data_Storage/Pretrained_Models'…

Output()

## Pipeline Selection

This section allows you to switch different pipelines for training and testing later. By default, TSU pipeline is selected. You can choose other pipeline from the dropdown list below to be used for the training and testing of your model.

You may proceed these pipelines listed below by clicking the link.
- [TSU Pipeline](#TSU)
- [STEP Pipeline](#STEP)

In [2]:
pipelineList = ["TSU", "STEP"]
pipeline_dropdown = widgets.Dropdown(
    options = pipelineList,
    default = 0,
    description = "Pipeline: "
)
confirmBtn = widgets.Button(
    description='Confirm Selection',
    tooltip='Confirm',
    button_style='success',
)
output = widgets.Output()

# Confirm Pipeline Selection
@output.capture(clear_output=True)
def confirm_pipeline(b):
    if pipeline_dropdown.value is not None:
        if pipeline_dropdown.value == "TSU":
            with output:
                print(f"{pipeline_dropdown.value} pipeline is selected")
                display(HTML('''
                    <a href="#TSU">Please proceed to TSU section</a>
                '''))
        elif pipeline_dropdown.value == "STEP":
            link = "#STEP"
            with output:
                print(f"{pipeline_dropdown.value} pipeline is selected")
                display(HTML('''
                    <a href="#STEP">Please proceed to STEP section</a>
                '''))
        
display(pipeline_dropdown)
display(confirmBtn, output)
confirmBtn.on_click(confirm_pipeline)

Dropdown(description='Pipeline: ', options=('TSU', 'STEP'), value='TSU')

Button(button_style='success', description='Confirm Selection', style=ButtonStyle(), tooltip='Confirm')

Output()

## TSU Pipeline <a class="anchor" id="TSU"></a>

## Training a model

### The following codes will allow the user to train a ML model.

#### Set values for training

- Before proceeding to set epochs and batch size for training, please ensure that you have select at least one inference model from the previous cell.

In [5]:
#Imports
import ipywidgets as widgets
from IPython.display import clear_output

trainBool = False

# UI for batch size
batchsize_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=4,
    step=1,
    description='Batch size:',
    continuous_update=False
)

# UI for epoch
epoch_slider = widgets.IntSlider(
    value=1000,
    min=0,
    max=1000,
    step=10,
    description='Epochs:',
    continuous_update=False
)

# Run training button
run_training_model = widgets.Button(
    description='Start Training',
    tooltip='Run',
    button_style='primary',
)

if ptmodel.selected_filename is not None:
    model_src = './Data_Storage/Pretrained_Models/' + ptmodel.selected_filename
    
    # Start Training function
    def start_training(b):
        clear_output(wait=False)
        print("\n Starting the process...")
        run = f"python TSU/test.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train False -num_channel 512 -lr 0.0002 -kernelsize 3 -APtype map -batch_size {str(batchsize_slider.value)} -comp_info TSU_CS_RGB_PDAN -load_model {model_src}"
        !{run}

    print("Epochs and Batch size selection:")
    display(epoch_slider)
    display(batchsize_slider)    
    display(run_training_model)

    run_training_model.on_click(start_training)

else:
    display(HTML('''
    <h3 style="color:red;">Error: Please select a model from the previous cell.</h3>'''))


 Starting the process...
PDAN
batch_size: 1
cuda_avail False
100%|████████████████████████████████████████| 536/536 [00:02<00:00, 222.95it/s]
Traceback (most recent call last):
  File "/Users/junjie/Documents/3104/new_project/ict3104-team16-2022/TSU/test.py", line 362, in <module>
    prob_val, val_loss, val_map, actAccuracy,activityIndexes = val_step(model, 0, dataloaders['val'], 0) 
  File "/Users/junjie/Documents/3104/new_project/ict3104-team16-2022/TSU/test.py", line 322, in val_step
    outputs, loss, probs, err = run_network(model, data, gpu, epoch)
  File "/Users/junjie/Documents/3104/new_project/ict3104-team16-2022/TSU/test.py", line 185, in run_network
    inputs = Variable(inputs.cuda(gpu))
  File "/Users/junjie/opt/miniconda3/envs/ict3104/lib/python3.10/site-packages/torch/cuda/__init__.py", line 221, in _lazy_init
    raise AssertionError("Torch not compiled with CUDA enabled")
AssertionError: Torch not compiled with CUDA enabled


### Renaming the Trained Models

This section allows you to choose a trained model and rename it.

#### 1. Select Trained Model
- Select a trained model from the Trained_Model folder.

In [6]:
selectedModelLoc = ""

# FileChooser for trained model
fcModelRN = FileChooser('./Data_Storage/Captions')
fcModelRN.sandbox_path = './Data_Storage/Captions'
fcModelRN.show_hidden = False
fcModelRN.filter_pattern = ['*']
fcModelRN.show_only_dirs = False
fcModelRN.title = '<b>Select File to rename</b>'


changeName_btn = widgets.Button(
    description='Start Training',
    tooltip='Run',
)

# callback function
def getName(chooser):
    selectedModelLoc = fcModelRN.selected_path+fcModelRN.selected_filename

display(fcModelRN)
fcModelRN.register_callback(getName)


FileChooser(path='/Users/junjie/Documents/3104/new_project/ict3104-team16-2022/Data_Storage/Captions', filenam…

#### 2. Rename selected trained model
- Input the name that you wants to change for the selected trained model.
- Click on the save button to change the name of the model.

In [7]:
modelNN = widgets.Text(
        placeholder='New Model Name',
        description='Name: ',
        disabled=False
    )
saveButton = widgets.Button(
        description='Save',
        disabled=False,
        button_style='success',
    )

# callback function
def changeName(chooser):
    if fcModelRN.selected_path is not None:
        newModelLoc = fcModelRN.selected_path + modelNN.value
        selectedModelLoc = fcModelRN.selected_path+fcModelRN.selected_filename
        if os.path.exists(newModelLoc):
             print("File name has already been used!")
        elif os.path.exists(selectedModelLoc):
            os.rename( selectedModelLoc, newModelLoc)
            print(f"The model has been renamed from {fcModelRN.selected_filename} to {modelNN.value}" )
        else:
             display(HTML('''
        <h3 style="color:red;">Error: Please re-select the file to rename at the previous cell.</h3>'''))
    else:
         display(HTML('''
        <h3 style="color:red;">Error: Please select a file to rename at the previous cell.</h3>'''))

saveButton.on_click(changeName)
display(modelNN)
display(saveButton)

Text(value='', description='Name: ', placeholder='New Model Name')

Button(button_style='success', description='Save', style=ButtonStyle())

The model has been renamed from gg to SS


## STEP Pipeline <a class="anchor" id="STEP"></a>